In [13]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from sklearn.decomposition import LatentDirichletAllocation, NMF 
from wordcloud import WordCloud, ImageColorGenerator 
import matplotlib.pyplot as plt 
import seaborn as sns
import csv

pd.options.mode.chained_assignment = None 
pd.set_option('display.max_colwidth', 100)

In [6]:
path = "chromedriver"
browser = webdriver.Chrome(path)

link = "https://www.yelp.com/biz/caff%C3%A8-panna-new-york?osq=panna+caffe"
browser.get(link)

time.sleep(2)

In [7]:
def check_exists_by_xpath(xpath):
    try:
        browser.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [8]:
page_source = browser.page_source
page_num = 10

reviewText = []
reviewRating = []
reviewID = []

for i in range(0, page_num):
    # parse to a soup
    page_source = browser.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    reviews_content = soup.find_all('div', class_="review__373c0__3MsBX border-color--default__373c0__1WKlL")
    for review in reviews_content:
        review_ID = review.find('a', class_="css-166la90").text
        review_Text = review.find('span', class_='raw__373c0__tQAx6').text
        review_Rating = review.find('span',class_='display--inline__373c0__1gaV4 border-color--default__373c0__1yxBb').div['aria-label'][0]
        # append to our accumulative lists
        reviewText.append(review_Text)
        reviewID.append(review_ID)
        reviewRating.append(review_Rating)
    # use selenium to go to the next page
    if (check_exists_by_xpath('//a[@class="next-link navigation-button__373c0__1f-Ah css-166la90"]')):
        browser.find_element_by_xpath('//a[@class="next-link navigation-button__373c0__1f-Ah css-166la90"]').click()
        time.sleep(1)
    

In [9]:
data = {
    'reviewText':reviewText,
    'reviewID':reviewID,
    'reviewRating':reviewRating
}

test = pd.DataFrame(data)

In [10]:
test.to_csv('Yelp.csv')

In [14]:
csvFile = open('Yelp.csv', 'r')
reader = csv.reader(csvFile)